Test NB

In [15]:
import asyncio
import json
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent, UserProxyAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from agents.prompts.unstructured_text_parser_message import UNSTRUCTURED_TEXT_PARSER_SYSTEM_MESSAGE
from autogen_ext.models.openai import OpenAIChatCompletionClient
from typing import List, Dict
import pandas as pd
import json
import pypdf
from dotenv import load_dotenv
import os


In [ ]:
load_dotenv()

In [17]:

def extract_text_from_pdf(file_path: str) -> str:
    """
    Extracts all text content from a given PDF file.

    This tool's purpose is to provide the raw text data from a PDF to an LLM agent.
    The agent will then be responsible for writing code to parse this unstructured text.

    Args:
        file_path (str): The local path to the PDF file.

    Returns:
        str: A single string containing all the extracted text from the PDF.
             Returns an error message string if extraction fails.
    """
    try:
        reader = pypdf.PdfReader(file_path)
        full_text = ""
        for page in reader.pages:
            full_text += page.extract_text() + "\n--- End of Page ---\n"
        return full_text
    except Exception as e:
        return f"Error extracting text from PDF: {str(e)}"


In [18]:
parsed_text = extract_text_from_pdf('temp/test_statement2.pdf')

In [ ]:
import pypdf
import os
from typing import List

def convert_pdfs_in_dir(input_dir: str, output_dir: str = "temp") -> List[str]:
    """
    Scans a directory for PDF files, extracts text, and saves each to a text file.

    This function searches the specified input directory for any files with a '.pdf'
    extension. It then extracts the text from each PDF and saves it to a new file 
    in the output directory. The output files are named sequentially 
    (e.g., statement1.txt, statement2.txt).

    Args:
        input_dir (str): The path to the directory containing the PDF files.
        output_dir (str): The name of the directory to save the text files.
                          Defaults to 'temp'.

    Returns:
        List[str]: A list of paths to the newly created text files.
                   Returns an empty list if the input directory doesn't exist
                   or contains no PDF files.
    """
    # Check if the input directory exists
    if not os.path.isdir(input_dir):
        print(f"Error: Input directory '{input_dir}' not found.")
        return []

    # Find all files in the directory that end with .pdf (case-insensitive)
    pdf_files = [
        os.path.join(input_dir, filename)
        for filename in os.listdir(input_dir)
        if filename.lower().endswith(".pdf") and os.path.isfile(os.path.join(input_dir, filename))
    ]

    if not pdf_files:
        print(f"No PDF files found in '{input_dir}'.")
        return []

    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    created_text_files = []

    # Enumerate through the list of discovered PDFs to process them
    for i, file_path in enumerate(pdf_files, start=1):
        try:
            print(f"Processing '{file_path}'...")
            reader = pypdf.PdfReader(file_path)
            full_text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    full_text += page_text + "\n"
            
            # Define the output file name and path
            output_filename = f"statement{i}.txt"
            output_path = os.path.join(output_dir, output_filename)
            
            # Write the extracted text to the new file
            with open(output_path, "w", encoding="utf-8") as f:
                f.write(full_text)
            
            created_text_files.append(output_path)
            print(f"Successfully created '{output_path}'")

        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            
    return created_text_files

    

In [ ]:
source_directory = "temp"

# Call the function to convert all PDFs in the directory
saved_files = convert_pdfs_in_dir(source_directory)

print("\n--- Conversion Complete ---")
if saved_files:
    print("The following text files were created in the 'temp' directory:")
    for file in saved_files:
        print(f"- {file}")
else:
    print("No files were created.")

In [ ]:
# --- 1. Imports ---
# Standard library and Pydantic
import os
import json
from typing import List, Optional
from pydantic import BaseModel, Field

# Core AutoGen components with explicit paths
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_core.tools import Workbench

# --- 2. Define the Desired JSON Structure ---
# This defines the schema for each individual transaction.
class Transaction(BaseModel):
    cardholder: str = Field(description="The name of the cardholder who made the transaction.")
    sale_date: Optional[str] = Field(description="The date the transaction was made, e.g., '07/15'")
    post_date: str = Field(description="The date the transaction was posted to the account, e.g., '07/16'")
    description: str = Field(description="The description of the transaction")
    transaction_amount: float = Field(description="The amount. Debits (purchases) must be negative, credits (payments) must be positive.")

# This is the main structure for the entire statement.
class StatementData(BaseModel):
    bank_name: str = Field(description="The name of the bank or card issuer, e.g., 'Costco Anywhere Visa Card by Citi'")
    transactions: List[Transaction] = Field(description="A list of all transactions from the statement")

# --- 3. Create the Python Function (The 'Tool') ---
def parse_bank_statement(bank_name: str, transactions: List[dict]) -> str:
    """
    Parses raw bank statement data into a structured JSON format using Pydantic models.
    
    Args:
        bank_name: The name of the bank or card issuer.
        transactions: A list of dictionaries, each representing a transaction.
                      Each dictionary must contain all fields for the Transaction model.
                      
    Returns:
        A JSON string representation of the structured data.
    """
    statement = StatementData(
        bank_name=bank_name,
        transactions=[Transaction(**t) for t in transactions]
    )
    return statement.model_dump_json(indent=2)

# --- 4. Configure AutoGen Agents and Environment ---
# Ensure your OPENAI_API_KEY is set as an environment variable
config_list = [
    {
        "model": "gpt-4o", 
        "api_key": os.getenv("OPENAI_API_KEY2")
    }
]

# The Workbench manages the tools and execution environment.
workbench = Workbench(work_dir="coding", tools=[parse_bank_statement])

# The UserProxyAgent acts as the executor.
user_proxy = UserProxyAgent(
    name="UserProxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    workbench=workbench,
)

# The AssistantAgent acts as the planner, deciding to use the tool.
parser_assistant = AssistantAgent(
    name="ParserAssistant",
    system_message="""You are an expert financial data extractor. Your task is to meticulously analyze the user's text and use the `parse_bank_statement` tool to convert it into a structured JSON format.

    CRITICAL INSTRUCTIONS:
    1.  **Cardholders:** The statement has multiple cardholders (e.g., MOHIT AGGARWAL, HIMANI SOOD). You MUST associate each transaction with the correct cardholder name it's listed under.
    2.  **Transaction Amounts:** You MUST correctly interpret the signs. 
        -   Purchases are DEBITS and must be represented as **NEGATIVE** numbers.
        -   Payments and Credits are CREDITS and must be represented as **POSITIVE** numbers. If a payment is shown as '-$1,041.44', its value is 1041.44.
    3.  **Data Focus:** Ignore all summary data, ads, addresses, and page markers. Focus ONLY on the detailed list of transactions.""",
    llm_config={"config_list": config_list},
    tools=[parse_bank_statement],
)

# --- 5. Your Input String is Loaded Here ---
pdf_text_string = parsed_text

# --- 6. Initiate the Conversation and Print the Final Output ---
user_proxy.initiate_chat(
    recipient=parser_assistant,
    message=f"Please parse the following bank statement text:\n\n{pdf_text_string}"
)

final_message = user_proxy.last_message(parser_assistant)
final_json_output = final_message.get("content", "")

print("\n--- Final Parsed JSON Output ---")
print(final_json_output)

In [13]:
# Install (run once)

# ---------------------------
# Agent + Code-executor demo
# ---------------------------
import os, asyncio
from autogen_agentchat.agents import AssistantAgent, CodeExecutorAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.messages import TextMessage

async def run_parse_v0_7_2(statement_text: str):
    # 1) model client used by AssistantAgent
    model_client = OpenAIChatCompletionClient(
        model="gpt-4o", 
        api_key=os.getenv("OPENAI_API_KEY2")
    )

    # 2) assistant - this agent will generate parsing code
    assistant = AssistantAgent(
        name="assistant",
        model_client=model_client,
        system_message=(
            "You are an expert Python developer. Given a raw multi-line bank statement string, "
            "produce Python code inside triple-backtick ```python``` blocks that parses the statement "
            "into a JSON object with keys: account_info, transactions, rewards and also prints a pandas DataFrame "
            "for transactions. When done, print the JSON and then reply with the single word: TERMINATE"
        ),
        reflect_on_tool_use=True,
    )

    # 3) code executor component (local) — start it before using
    code_executor = LocalCommandLineCodeExecutor(work_dir="coding")
    await code_executor.start()

    # 4) code executor *agent* (do NOT pass llm_config)
    code_executor_agent = CodeExecutorAgent(
        name="code_executor",
        code_executor=code_executor,
        # optional: model_client=..., approval_func=..., supported_languages=...
        approval_func=None,
    )

    # 5) team: assistant -> code executor (round-robin). Terminate on 'TERMINATE' or after N messages
    termination = TextMentionTermination("TERMINATE") | MaxMessageTermination(6)
    team = RoundRobinGroupChat(participants=[assistant, code_executor_agent], termination_condition=termination)

    # 6) the task: feed the raw statement and ask for JSON + DataFrame
    task_msg = TextMessage(
        content=(
            "Parse the bank statement below into JSON & a pandas DataFrame. "
            "Print the JSON first, then print the DataFrame. End your final response with TERMINATE.\n\n"
            f"{statement_text}"
        ),
        source="user",
    )

    # 7) run the team (synchronous top-level via asyncio)
    result = await team.run(task=task_msg)
    # final model text available as result.chat_message
    print("===== AGENT FINAL OUTPUT =====")
    print(result.chat_message.content)

    # cleanup
    await code_executor.stop()



In [ ]:
# Usage: replace the string below with your extracted-statement string
YOUR_STATEMENT = parsed_text
asyncio.run(run_parse_v0_7_2(YOUR_STATEMENT))